## Semiprime factors ratio detection

This test is based on Sam Blake's preprint "Integer Factorisation, Fermat & 
Machine Learning on a Classical Computer", arXiv:2308.12290

Detecting the ratio of semiprime factors may theoretically help improving 
the classical Lawrence algorithm for semiprime factorization

### Imports

In [13]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import sys
sys.path.insert(0, '../..')

In [3]:
import numpy as np
import gmpy2
from gmpy2 import mpz, mpq, mpfr
from cerebros.simplecerebrosrandomsearch.simple_cerebros_random_search\
    import SimpleCerebrosRandomSearch
import pendulum
import pandas as pd
import tensorflow as tf
from cerebros.units.units import DenseUnit
from cerebros.denseautomlstructuralcomponent.dense_automl_structural_component\
    import zero_7_exp_decay, zero_95_exp_decay, simple_sigmoid
from ast import literal_eval
from sklearn.metrics import confusion_matrix

/Users/sasha/jax-metal/lib/python3.8/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### Create and run Cerebros session

Detecting if a given 128-bit semiprime has factor ratio between 2 and 3

In [15]:
raw_data = pd.read_csv("semiprime_ratio_training_data_128_ratio_2_3.csv")
raw_data.iloc[:,0] = raw_data.iloc[:,0].apply(mpz)
raw_data = raw_data.values
X = np.array(list(map(lambda x: [int(i) for i in gmpy2.digits(x,2)], raw_data[:,0])), dtype=np.int32)
y = np.array(raw_data[:,1], dtype=np.int32)

Use only a relatively small number of datapoints for training, let's say 20% of the whole dataset. The remainig 80% is for testing. 

In [16]:
num_data_points = 200_000
X_train, y_train = X[:num_data_points,:], y[:num_data_points]
X_test, y_test = X[num_data_points:,:], y[num_data_points:]

In [17]:
tensor_x = tf.constant(X_train)
training_x = [tensor_x]
INPUT_SHAPES = [training_x[i].shape[1] for i in np.arange(len(training_x))]
train_labels = [y_train]
OUTPUT_SHAPES = [1] 

Cerebros AutoML search parameters

In [18]:
# Logging
TIME = pendulum.now().__str__()[:16]\
    .replace('T', '_')\
    .replace(':', '_')\
    .replace('-', '_')
PROJECT_NAME = f'{TIME}_cerebros_auto_ml_test_semiprimes'

# Architecture and training hyperparameters
#
meta_trial_number = 42  # in distributed training set this to a random number
#
activation = "relu"
predecessor_level_connection_affinity_factor_first = 19.613
predecessor_level_connection_affinity_factor_main = 0.5518
max_consecutive_lateral_connections = 34
p_lateral_connection = 0.36014
num_lateral_connection_tries_per_unit = 11
learning_rate = 0.095
epochs = 10
batch_size = 634
maximum_levels = 5
maximum_units_per_level = 5
maximum_neurons_per_unit = 25

Instantiate a search session

In [19]:
#
cerebros_automl =\
    SimpleCerebrosRandomSearch(
        unit_type=DenseUnit,
        input_shapes=INPUT_SHAPES,
        output_shapes=OUTPUT_SHAPES,
        training_data=training_x,
        labels=train_labels,
        validation_split=0.35,
        direction='maximize',
        metric_to_rank_by='val_binary_accuracy',
        minimum_levels=1,
        maximum_levels=maximum_levels,
        minimum_units_per_level=1,
        maximum_units_per_level=maximum_units_per_level,
        minimum_neurons_per_unit=1,
        maximum_neurons_per_unit=maximum_neurons_per_unit,
        activation=activation,
        final_activation='sigmoid',
        number_of_architecture_moities_to_try=3,
        number_of_tries_per_architecture_moity=2, #if I put 1 here, it bugs out !!!
        number_of_generations=3,
        minimum_skip_connection_depth=1,
        maximum_skip_connection_depth=7,
    predecessor_level_connection_affinity_factor_first=predecessor_level_connection_affinity_factor_first,
        predecessor_level_connection_affinity_factor_first_rounding_rule='ceil',
            predecessor_level_connection_affinity_factor_main=predecessor_level_connection_affinity_factor_main,
        predecessor_level_connection_affinity_factor_main_rounding_rule='ceil',
        predecessor_level_connection_affinity_factor_decay_main=zero_7_exp_decay,
        seed=8675309,
        max_consecutive_lateral_connections=max_consecutive_lateral_connections,
        gate_after_n_lateral_connections=3,
        gate_activation_function=simple_sigmoid,
        p_lateral_connection=p_lateral_connection,
        p_lateral_connection_decay=zero_95_exp_decay,
        num_lateral_connection_tries_per_unit=num_lateral_connection_tries_per_unit,
        learning_rate=learning_rate,
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy(), tf.keras.metrics.TrueNegatives(),
                 tf.keras.metrics.FalseNegatives(), tf.keras.metrics.FalsePositives(),
                 tf.keras.metrics.TruePositives()],
        epochs=epochs,
        patience=7,
        project_name=f"{PROJECT_NAME}_meta_{meta_trial_number}",
        model_graphs='model_graphs',
        batch_size=batch_size,
        meta_trial_number=meta_trial_number)
#

Running search

In [20]:
result = cerebros_automl.run_random_search()
best_model_found = cerebros_automl.get_best_model()

SimpleCerebrosRandomSearch.input_shapes: [128]
nan
>nnf>ceil
k is: 0 value is: [{'1': <class 'cerebros.units.units.InputUnit'>}]
0
k is: 1 value is: [{'6': <class 'cerebros.units.units.DenseUnit'>}, {'11': <class 'cerebros.units.units.DenseUnit'>}]
1
Trying to create level 1
We think level 1's predecessors are: [0]
k is: 2 value is: [{'1': <class 'cerebros.units.units.FinalDenseUnit'>}]
2
Trying to create Final level 2
Trying to create level 2
We think level final level 2's predecessors are: [0, 1]
levels:
[0, 1, 2]
{'0': 'InputUnitModule'}
InputLevel.input_shapes [128]
{'6': <class 'cerebros.units.units.DenseUnit'>}
{'11': <class 'cerebros.units.units.DenseUnit'>}
Debug: I am 2 selecting 1
debug: meta_level_number
debug: meta_level_number
debug: meta_level_number
Setting levels_unmaterialized[0] level_number 0 to have first successor: levels_unmaterialized[:1], having level_numbers of [1, 2]
Setting levels_unmaterialized[1] level_number 1 to have first successor: levels_unmaterialized

Debug: successor_connectivity_errors_2d []
materialize:_NeuralNetworkFuture_0000000000000nan_tr_0_FinalDenseLevel_0000000000000002_tr_0_FinalDenseUnit_0000000000000002_tr_0_0 called
materialized network layers
[<KerasTensor: shape=(None, 6) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000001_tr_0_DenseUnit_0000000000000001_tr_0_0_dns_')>, <KerasTensor: shape=(None, 6) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000001_tr_0_DenseUnit_0000000000000001_tr_0_0_dns_')>, <KerasTensor: shape=(None, 6) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000001_tr_0_DenseUnit_0000000000000001_tr_0_0_dns_')>, <KerasTensor: shape=(None, 6) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000001_tr_0_DenseUnit_0000000000000001_tr_0_0_dns_')>, <KerasTensor: shape=(None, 128) dtype=float32 (created by layer '

INFO:tensorflow:Assets written to: 2023_10_19_11_45_cerebros_auto_ml_test_semiprimes_meta_42/models/tr_0000000000000000_subtrial_0000000000000000/assets


returning trial 0 oracles
       loss  binary_accuracy  true_negatives_1  false_negatives_1  \
0  0.546904         0.707185           39245.0            12638.0   
1  0.526507         0.714562           38948.0            11382.0   
2  0.527469         0.712985           38976.0            11615.0   
3  0.528430         0.713208           39206.0            11816.0   
4  0.529826         0.711738           39092.0            11893.0   
5  0.529934         0.711462           39146.0            11983.0   
6  0.529720         0.712092           38982.0            11737.0   
7  0.533823         0.708185           39197.0            12460.0   
8  0.529966         0.711008           39013.0            11909.0   
9  0.534949         0.707277           38908.0            12289.0   

   false_positives_1  true_positives_1  val_loss  val_binary_accuracy  \
0            25428.0           52689.0  0.559302             0.652429   
1            25725.0           53945.0  0.525827             0.70557

INFO:tensorflow:Assets written to: 2023_10_19_11_45_cerebros_auto_ml_test_semiprimes_meta_42/models/tr_0000000000000000_subtrial_0000000000000001/assets


...

materialize:_NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000002_tr_0_DenseUnit_0000000000000002_tr_0_3 called
materialized network layers
[<KerasTensor: shape=(None, 9) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000002_tr_0_DenseUnit_0000000000000002_tr_0_2_dns_')>, <KerasTensor: shape=(None, 9) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000002_tr_0_DenseUnit_0000000000000002_tr_0_2_dns_')>, <KerasTensor: shape=(None, 9) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000002_tr_0_DenseUnit_0000000000000002_tr_0_2_dns_')>, <KerasTensor: shape=(None, 9) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000002_tr_0_DenseUnit_0000000000000002_tr_0_2_dns_')>, <KerasTensor: shape=(None, 9) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_00

INFO:tensorflow:Assets written to: 2023_10_19_11_45_cerebros_auto_ml_test_semiprimes_meta_42/models/tr_0000000000000000_subtrial_0000000000000000/assets


returning trial 0 oracles
       loss  binary_accuracy  true_negatives_1  false_negatives_1  \
0  0.575688         0.708700           59006.0            17460.0   
1  0.524399         0.716677           39014.0            11173.0   
2  0.526799         0.714031           38980.0            11483.0   
3  0.530223         0.711662           39259.0            12070.0   
4  0.530604         0.710977           39149.0            12049.0   
5  0.537781         0.705408           39442.0            13066.0   
6  0.537114         0.706938           39143.0            12568.0   
7  0.532503         0.711077           38299.0            11186.0   
8  0.532577         0.711200           37345.0            10216.0   
9  0.529575         0.713462           37111.0             9688.0   

   false_positives_1  true_positives_1  val_loss  val_binary_accuracy  \
0            40800.0           82734.0  0.541102             0.696400   
1            25659.0           54154.0  0.536229             0.71117

INFO:tensorflow:Assets written to: 2023_10_19_11_45_cerebros_auto_ml_test_semiprimes_meta_42/models/tr_0000000000000000_subtrial_0000000000000001/assets


returning trial 0 oracles
       loss  binary_accuracy  true_negatives_1  false_negatives_1  \
0  0.581126         0.704585           61519.0            20796.0   
1  0.520320         0.720069           37083.0             8801.0   
2  0.523918         0.718800           36714.0             8597.0   
3  0.524814         0.715954           37074.0             9327.0   
4  0.527889         0.716692           37408.0             9565.0   
5  0.526117         0.716831           36913.0             9052.0   
6  0.528727         0.713815           37171.0             9702.0   
7  0.531165         0.712654           37366.0            10048.0   
8  0.533638         0.709862           37503.0            10548.0   
9  0.528213         0.714831           37278.0             9677.0   

   false_positives_1  true_positives_1  val_loss  val_binary_accuracy  \
0            38287.0           79398.0  0.551279             0.657957   
1            27590.0           56526.0  0.517649             0.72087

Output best model parameters and accuracy

In [21]:
trainable_params = np.sum([np.prod(w.get_shape()) for w in best_model_found.trainable_weights])
non_trainable_params = np.sum([np.prod(w.get_shape()) for w in best_model_found.non_trainable_weights])
total_params = trainable_params + non_trainable_params

print(f"Best model found: {total_params} total parameters ({trainable_params} trainable, {non_trainable_params} non-trainable)")

print(f"Best accuracy is ({cerebros_automl.metric_to_rank_by}): {result}")

Best model found: 141511 total parameters (127649 trainable, 13862 non-trainable)
Best accuracy is (val_binary_accuracy): 0.7237714529037476


Make predictions on the remaining 90% of the dataset

In [22]:
best_model_found.compile()
best_model_found.summary()

Model: "NeuralNetworkFuture_0000000000000nan_tr_0_nn_materialized"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 NeuralNetworkFuture_000000  [(None, 128)]                0         []                            
 0000000nan_tr_0_InputLevel                                                                       
 _0000000000000000_tr_0_Inp                                                                       
 utUnit_0000000000000000_tr                                                                       
 _0_0_inp (InputLayer)                                                                            
                                                                                                  
 NeuralNetworkFuture_000000  (None, 5120)                 0         ['NeuralNetworkFuture_00000000
 0000000nan_tr_0_DenseLevel               

In [25]:
y_pred = best_model_found.predict(X_test)
threshold = y_pred.mean()
y_pred = (y_pred > threshold).astype(int)  

25000/25000 [==============================] - 48s 2ms/step


Confusion matrix

In [26]:
cm = confusion_matrix(y_test, y_pred, normalize='all')
cm

array([[0.3020375 , 0.19820625],
       [0.08584   , 0.41391625]])

Blake's confusion matrix for comparison


  0.273  0.227
---------------
  0.056  0.444
---------------

In our case, we get ~3% more false negatives. However, we get ~10% better accuracy. Given that we used only 20% of the dataset for training, and 80% for testing, this result looks good.